# Imports

In [1]:
import numpy as np
import math
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from scipy.stats import norm

# Setup

In [2]:
def maxnorm(elo1, elo2, var=1524):
    #elo1+=var/10
    nm=elo1-elo2
    var*=2
    return 1-norm.cdf(-nm/var**.5)

In [3]:
def recalib(forecast): # 2000 looks best
    return forecast -np.sign(2*(forecast-.5))*.08*abs(2*(forecast-.5))**(1./51)+.04*(2*(forecast-.5))**3
#- .1*(2*(forecast-.5))**1 -.04*(2*(forecast-.5))**5+.06*(2*(forecast-.5))**3
#- .015*(forecast-.5)**5 - .01*(forecast-.5)**3 - .03*(forecast-.5) -.02 *(forecast-.5)**4     
#forecast+.057*(2*(forecast-.5))**201
#forecast+.06*(2*(forecast-.5))**451-.059*(2*(forecast-.5))**27 with a=.1     

In [4]:
NBA_data= pd.read_csv("nbaallelo.csv")
NBA_data

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,40.294830,NYK,Knicks,68,1300.0000,1306.7233,H,L,0.640065,NaN
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,41.705170,TRH,Huskies,66,1300.0000,1293.2767,A,W,0.359935,NaN
2,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,42.012257,NYK,Knicks,47,1306.7233,1297.0712,H,W,0.631101,NaN
3,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,40.692783,CHS,Stags,63,1300.0000,1309.6521,A,L,0.368899,NaN
4,3,194611020DTF,NBA,0,1947,11/2/1946,1,0,DTF,Falcons,...,38.864048,WSC,Capitols,50,1300.0000,1320.3811,H,L,0.640065,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126309,63155,201506110CLE,NBA,0,2015,6/11/2015,100,1,CLE,Cavaliers,...,60.309792,GSW,Warriors,103,1790.9591,1809.9791,H,L,0.546572,NaN
126310,63156,201506140GSW,NBA,0,2015,6/14/2015,102,1,GSW,Warriors,...,68.013329,CLE,Cavaliers,91,1704.3949,1700.7391,H,W,0.765565,NaN
126311,63156,201506140GSW,NBA,1,2015,6/14/2015,101,1,CLE,Cavaliers,...,60.010067,GSW,Warriors,104,1809.9791,1813.6349,A,L,0.234435,NaN
126312,63157,201506170CLE,NBA,0,2015,6/16/2015,102,1,CLE,Cavaliers,...,59.290245,GSW,Warriors,105,1813.6349,1822.2881,H,L,0.481450,NaN


In [5]:
NBA_data.columns

Index(['gameorder', 'game_id', 'lg_id', '_iscopy', 'year_id', 'date_game',
       'seasongame', 'is_playoffs', 'team_id', 'fran_id', 'pts', 'elo_i',
       'elo_n', 'win_equiv', 'opp_id', 'opp_fran', 'opp_pts', 'opp_elo_i',
       'opp_elo_n', 'game_location', 'game_result', 'forecast', 'notes'],
      dtype='object')

In [6]:
NBA_data['maxnorm_forecast'] = NBA_data.apply(lambda x: maxnorm(x['elo_i'], x['opp_elo_i']), axis=1)
NBA_data['results'] = (NBA_data['game_result'] == 'W')*1

In [7]:
NBA_var=np.var(NBA_data['elo_i'])
NBA_mean=np.mean(NBA_data[NBA_data['team_id']=='NYK']['elo_i'])

print(NBA_mean, NBA_var/(2015-1947)**.5) #/(126314**.5)

1497.6156040908302 1524.9752090829077


In [8]:
NBA_data[['elo_i', 'opp_elo_i', 'forecast', 'maxnorm_forecast', 'results']]

,elo_i,opp_elo_i,forecast,maxnorm_forecast,results
0,1300.0000,1300.0000,0.640065,0.500000,0
1,1300.0000,1300.0000,0.359935,0.500000,1
2,1300.0000,1306.7233,0.631101,0.451537,1
3,1306.7233,1300.0000,0.368899,0.548463,0
4,1300.0000,1300.0000,0.640065,0.500000,0
...,...,...,...,...,...
126309,1723.4149,1790.9591,0.546572,0.110583,0
126310,1809.9791,1704.3949,0.765565,0.972091,1
126311,1704.3949,1809.9791,0.234435,0.027909,0
126312,1700.7391,1813.6349,0.481450,0.020433,0


In [9]:
regdf1=NBA_data[['game_id','forecast', 'maxnorm_forecast']]
model1 = smf.ols(formula='maxnorm_forecast ~ forecast', data=regdf1).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       maxnorm_forecast   R-squared:                       0.585
Model:                            OLS   Adj. R-squared:                  0.585
Method:                 Least Squares   F-statistic:                 1.777e+05
Date:                Tue, 05 Jan 2021   Prob (F-statistic):               0.00
Time:                        16:25:23   Log-Likelihood:                -11733.
No. Observations:              126314   AIC:                         2.347e+04
Df Residuals:                  126312   BIC:                         2.349e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2316      0.002   -122.574      0.000      -0.235      -0.228
forecast       1.4632      0.003    421.564      0.000       1.456       1.470
==============================================================================
Omnibus:                     7273.185   Durbin-Watson:                   2.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2712.335
Skew:                          -0.000   Prob(JB):                         0.00
Kurtosis:                       2.282   Cond. No.                         5.85
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# 538 Model Test Results

In [10]:
regdf2=NBA_data[['game_id','forecast', 'results']]
model2 = smf.ols(formula='forecast ~ results', data=regdf2).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               forecast   R-squared:                       0.185
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                 2.864e+04
Date:                Tue, 05 Jan 2021   Prob (F-statistic):               0.00
Time:                        16:25:23   Log-Likelihood:                 27686.
No. Observations:              126314   AIC:                        -5.537e+04
Df Residuals:                  126312   BIC:                        -5.535e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4075      0.001    526.892      0.000       0.406       0.409
results        0.1851      0.001    169.227      0.000       0.183       0.187
==============================================================================
Omnibus:                     6278.891   Durbin-Watson:                   3.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2485.259
Skew:                          -0.000   Prob(JB):                         0.00
Kurtosis:                       2.313   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
log_reg1 = sm.Logit(regdf2['results'], regdf2['forecast']).fit() 
log_reg1.summary()

Optimization terminated successfully.
         Current function value: 0.678589
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                results   No. Observations:               126314
Model:                          Logit   Df Residuals:                   126313
Method:                           MLE   Df Model:                            0
Date:                Tue, 05 Jan 2021   Pseudo R-squ.:                 0.02100
Time:                        16:25:23   Log-Likelihood:                -85715.
converged:                       True   LL-Null:                       -87554.
Covariance Type:            nonrobust   LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
forecast       0.6341      0.011     59.958      0.000       0.613       0.655
==============================================================================
"""

# Weighted Mix

In [12]:
a=.3
NBA_data['rw_forecast']= recalib(NBA_data['maxnorm_forecast']*a+ NBA_data['forecast']*(1-a))

In [13]:
NBA_data

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes,maxnorm_forecast,results,rw_forecast
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,68,1300.0000,1306.7233,H,L,0.640065,NaN,0.500000,0,0.520862
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,66,1300.0000,1293.2767,A,W,0.359935,NaN,0.500000,1,0.479138
2,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,47,1306.7233,1297.0712,H,W,0.631101,NaN,0.451537,1,0.500256
3,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,63,1300.0000,1309.6521,A,L,0.368899,NaN,0.548463,0,0.499744
4,3,194611020DTF,NBA,0,1947,11/2/1946,1,0,DTF,Falcons,...,50,1300.0000,1320.3811,H,L,0.640065,NaN,0.500000,0,0.520862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126309,63155,201506110CLE,NBA,0,2015,6/11/2015,100,1,CLE,Cavaliers,...,103,1790.9591,1809.9791,H,L,0.546572,NaN,0.110583,0,0.492839
126310,63156,201506140GSW,NBA,0,2015,6/14/2015,102,1,GSW,Warriors,...,91,1704.3949,1700.7391,H,W,0.765565,NaN,0.972091,1,0.759427
126311,63156,201506140GSW,NBA,1,2015,6/14/2015,101,1,CLE,Cavaliers,...,104,1809.9791,1813.6349,A,L,0.234435,NaN,0.027909,0,0.240573
126312,63157,201506170CLE,NBA,0,2015,6/16/2015,102,1,CLE,Cavaliers,...,105,1813.6349,1822.2881,H,L,0.481450,NaN,0.020433,0,0.420112


In [14]:
regdf4=NBA_data[['game_id','rw_forecast', 'results','forecast']]
model4 = smf.ols(formula='rw_forecast ~ results', data=regdf4).fit()
model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            rw_forecast   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                  0.169
Method:                 Least Squares   F-statistic:                 2.567e+04
Date:                Tue, 05 Jan 2021   Prob (F-statistic):               0.00
Time:                        16:25:24   Log-Likelihood:                 34466.
No. Observations:              126314   AIC:                        -6.893e+04
Df Residuals:                  126312   BIC:                        -6.891e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4170      0.001    568.917      0.000       0.416       0.418
results        0.1661      0.001    160.217      0.000       0.164       0.168
==============================================================================
Omnibus:                     6368.159   Durbin-Watson:                   2.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2506.547
Skew:                          -0.000   Prob(JB):                         0.00
Kurtosis:                       2.310   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
log_reg4 = sm.Logit(regdf4['results'], regdf4['rw_forecast']).fit() 
log_reg4.summary()

Optimization terminated successfully.
         Current function value: 0.681222
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                results   No. Observations:               126314
Model:                          Logit   Df Residuals:                   126313
Method:                           MLE   Df Model:                            0
Date:                Tue, 05 Jan 2021   Pseudo R-squ.:                 0.01720
Time:                        16:25:24   Log-Likelihood:                -86048.
converged:                       True   LL-Null:                       -87554.
Covariance Type:            nonrobust   LLR p-value:                       nan
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
rw_forecast     0.5779      0.011     54.393      0.000       0.557       0.599
===============================================================================
"""

# Binned Testing 538

In [16]:
binsz=2
tplimit=1.0000000000001
bindff=NBA_data[['forecast','results']]
bins= np.arange(0,tplimit,binsz/100)
labels= np.arange(0,100,binsz)
print(bins)
bindff['binned']=pd.cut(bindff['forecast'],bins=bins,labels=labels)
bindff['binned']=bindff['binned'].fillna(0)
bindff

[0.   0.02 0.04 0.06 0.08 0.1  0.12 0.14 0.16 0.18 0.2  0.22 0.24 0.26
 0.28 0.3  0.32 0.34 0.36 0.38 0.4  0.42 0.44 0.46 0.48 0.5  0.52 0.54
 0.56 0.58 0.6  0.62 0.64 0.66 0.68 0.7  0.72 0.74 0.76 0.78 0.8  0.82
 0.84 0.86 0.88 0.9  0.92 0.94 0.96 0.98 1.  ]


C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,forecast,results,binned
0,0.640065,0,64
1,0.359935,1,34
2,0.631101,1,62
3,0.368899,0,36
4,0.640065,0,64
...,...,...,...
126309,0.546572,0,54
126310,0.765565,1,76
126311,0.234435,0,22
126312,0.481450,0,48


In [17]:
bin_compf=[]
for i in range(len(bins)-1):
    binaver= np.mean(bindff['results'][bindff['binned']==labels[i]])
    bin_compf.append([(bins[i]+bins[i+1])/2,binaver])
bin_compf

[[0.01, nan],
 [0.03, 0.046153846153846156],
 [0.05, 0.03873239436619718],
 [0.07, 0.06615384615384616],
 [0.09, 0.08739837398373984],
 [0.11, 0.10145985401459855],
 [0.13, 0.1357273280584297],
 [0.15000000000000002, 0.1506849315068493],
 [0.16999999999999998, 0.1735201084500678],
 [0.19, 0.19041718298223875],
 [0.21000000000000002, 0.21509710516672773],
 [0.22999999999999998, 0.23084922010398615],
 [0.25, 0.25208681135225375],
 [0.27, 0.26206673145448656],
 [0.29000000000000004, 0.2964062964062964],
 [0.31, 0.30960648148148145],
 [0.33, 0.3352365415986949],
 [0.35, 0.34360063728093465],
 [0.37, 0.36207359656191246],
 [0.39, 0.39315548369711667],
 [0.41000000000000003, 0.4130085653104925],
 [0.43, 0.4377431906614786],
 [0.45, 0.4518413597733711],
 [0.47, 0.4816538882803943],
 [0.49, 0.49739225912709306],
 [0.51, 0.5026077408729069],
 [0.53, 0.5183461117196057],
 [0.55, 0.5481586402266289],
 [0.5700000000000001, 0.5622568093385214],
 [0.59, 0.5869914346895075],
 [0.61, 0.606844516302883

# Weighted Binned Testing

In [18]:
bindfw=NBA_data[['rw_forecast','results']]
bins= np.arange(0,tplimit,binsz/100)
labels= np.arange(0,100,binsz)
print(bins)
bindfw['binned']=pd.cut(bindfw['rw_forecast'],bins=bins,labels=labels)
bindfw['binned']=bindfw['binned'].fillna(0)
bindfw

[0.   0.02 0.04 0.06 0.08 0.1  0.12 0.14 0.16 0.18 0.2  0.22 0.24 0.26
 0.28 0.3  0.32 0.34 0.36 0.38 0.4  0.42 0.44 0.46 0.48 0.5  0.52 0.54
 0.56 0.58 0.6  0.62 0.64 0.66 0.68 0.7  0.72 0.74 0.76 0.78 0.8  0.82
 0.84 0.86 0.88 0.9  0.92 0.94 0.96 0.98 1.  ]


C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,rw_forecast,results,binned
0,0.520862,0,52
1,0.479138,1,46
2,0.500256,1,50
3,0.499744,0,48
4,0.520862,0,52
...,...,...,...
126309,0.492839,0,48
126310,0.759427,1,74
126311,0.240573,0,24
126312,0.420112,0,42


In [19]:
bin_compw=[]
for i in range(len(bins)-1):
    binaver= np.mean(bindfw['results'][bindfw['binned']==labels[i]])
    bin_compw.append([(bins[i]+bins[i+1])/2,binaver])
bin_compw

[[0.01, nan],
 [0.03, nan],
 [0.05, 0.0],
 [0.07, 0.04032258064516129],
 [0.09, 0.04807692307692308],
 [0.11, 0.08546168958742632],
 [0.13, 0.09637814610190301],
 [0.15000000000000002, 0.1409068746952706],
 [0.16999999999999998, 0.15256257449344457],
 [0.19, 0.1927541329581428],
 [0.21000000000000002, 0.20020087043856713],
 [0.22999999999999998, 0.22218430034129694],
 [0.25, 0.2467387253074916],
 [0.27, 0.2625],
 [0.29000000000000004, 0.28888054094665666],
 [0.31, 0.31098696461824954],
 [0.33, 0.3245791245791246],
 [0.35, 0.36875439831104856],
 [0.37, 0.3738071734123067],
 [0.39, 0.4082164982067167],
 [0.41000000000000003, 0.4293324976496396],
 [0.43, 0.4612060094669685],
 [0.45, 0.4699853587115666],
 [0.47, 0.4955440029706647],
 [0.49, 0.48851030110935023],
 [0.51, 0.5114896988906498],
 [0.53, 0.5044559970293353],
 [0.55, 0.5300146412884333],
 [0.5700000000000001, 0.5387939905330315],
 [0.59, 0.5706675023503603],
 [0.61, 0.5917835017932833],
 [0.63, 0.6261928265876934],
 [0.65, 0.6312

# End